In [1]:
#Import Libraries
import cv2
import itertools
import numpy as np
import time
import mediapipe as mp
import re
import glob
import os
import pickle
import json

from tqdm import tqdm
from skimage import transform
import concurrent.futures

In [2]:
# Initialize the mediapipe face detection class
mp_face_detection = mp.solutions.face_detection
# Setup the face detection function
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
#initialize the mediapipe face mesh class
mp_face_mesh = mp.solutions.face_mesh
# Setup the face landmarks function for videos
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                         min_detection_confidence=0.5,min_tracking_confidence=0.3)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def save2npz(filename, data=None):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    np.savez_compressed(filename, data=data)

In [4]:
def get_wordslist_from_txt_file(file_path):
    with open(file_path) as file:
        word_list = file.readlines()
        word_list = [item.rstrip() for item in word_list]
    return word_list

In [5]:
def extract_points_from_mesh(face_landmarks, indexes):
    points_data_regex = re.compile(r'\d\.\d+')
    xy_points_list = []
    for count, each_index in enumerate(indexes):
        xyzpointsraw = face_landmarks.landmark[each_index]
        points_list = points_data_regex.findall(str(xyzpointsraw))
        if len(points_list) < 1:
            xy_points_list.append([None])
        else:
            xyclean = [float(points_list[0]), float(points_list[1])]
            xy_points_list.append(xyclean)
    xy_points_array = np.array(xy_points_list)
    return xy_points_array

In [6]:
def detect_Facial_Landmarks(image):
    frame_dict = {}
    mesh_result = face_mesh_videos.process(image)
    oval_indexes = list(set(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))
    lips_indexes = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
    if mesh_result.multi_face_landmarks:
        for face_no, face_landmarks in enumerate(mesh_result.multi_face_landmarks):
            oval_points_array = extract_points_from_mesh(face_landmarks, oval_indexes)
            lips_points_array = extract_points_from_mesh(face_landmarks, lips_indexes)
            frame_dict['oval_landmarks'] = oval_points_array
            frame_dict['lips_landmarks'] = lips_points_array
    else:
        frame_dict['oval_landmarks'] = None
        frame_dict['lips_landmarks'] = None
    return frame_dict

In [7]:
def get_face_points(video):
    vid_capture = cv2.VideoCapture(video)
    all_points = []
    if (vid_capture.isOpened() == False):
        print("Error opening the video file")
    else:
        while(vid_capture.isOpened()):
            ret, frame = vid_capture.read()
            if ret:
                frame_points_dict = detect_Facial_Landmarks(frame)
                all_points.append(frame_points_dict)
                k = cv2.waitKey(30)
                if(k == 27):
                    break
            else:
                break
    vid_capture.release()
    cv2.destroyAllWindows()
    return np.array(all_points)

In [8]:
def landmarks_extract_save(video, landmarks_output):
    dir, file = os.path.split(video)
    part, which_folder = os.path.split(dir)
    otherpart, word = os.path.split(part)
    new_filename = file[:-4] + ".npz"
    new_save_output = os.path.join(landmarks_output, word, which_folder, new_filename)
    if not os.path.exists(new_save_output):
        face_points_array = get_face_points(video)
        save2npz(new_save_output, face_points_array)

In [9]:
#Has issues with mediapipe and multiprocess
def multi_extract_landmarks(lrw_direc, landmarks_output):
    videos = glob.glob(os.path.join(lrw_direc, '*', which_folder, '*.mp4'))
    l= len(videos)
    with tqdm(total = l) as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(landmarks_extract_save, video, landmarks_output) for video in videos]
            for future in concurrent.futures.as_completed(futures):
                pbar.update(1)

In [10]:
def multi_batch_extract_landmarks(word_list, lrw_direc, which_folder, landmarks_output, workers=10, batch_split=4):
    for i in range(len(word_list)):
        print("Current Word: {}".format(word_list[i]))
        videos = glob.glob(os.path.join(lrw_direc, word_list[i], which_folder, '*.mp4'))
        chunk_size = int(len(videos)/batch_split)
        videos = [videos[f:f+chunk_size] for f in range(0, len(videos), chunk_size)]
        for pos in range(len(videos)):
            print("Batch {}/{}".format(pos+1, batch_split))
            batch = videos[pos]
            l=len(batch)
            with tqdm(total = l) as pbar:
                with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
                    futures = [executor.submit(landmarks_extract_save, video, landmarks_output) for video in batch]
                    for future in concurrent.futures.as_completed(futures):
                        pbar.update(1)
            time.sleep(2)

In [11]:
lrw_path = '/media/taylorpap/1TBM2/DatasetML/lipread_mp4'
landmarks_path = '/home/taylorpap/Bootcamp/LANDMARKS'
words_list_file = '/home/taylorpap/Bootcamp/wordlist.txt'
#which_folder = 'test'
which_folder = 'train'


In [12]:
words_list = get_wordslist_from_txt_file(words_list_file)

In [13]:
#multi_batch_extract_landmarks(words_list, lrw_path, which_folder, landmarks_path)

In [16]:
videos = glob.glob(os.path.join(lrw_path, '*', which_folder, '*.mp4'))
for video in tqdm(videos):
    landmarks_extract_save(video, landmarks_path)

100%|██████████| 25000/25000 [1:07:44<00:00,  6.15it/s]


In [17]:
which_folder = 'val'
videos = glob.glob(os.path.join(lrw_path, '*', which_folder, '*.mp4'))
for video in tqdm(videos):
    landmarks_extract_save(video, landmarks_path)

100%|██████████| 25000/25000 [1:08:31<00:00,  6.08it/s]


In [15]:
#temp_words_list = ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT', 'WEATHER']
# Extra/Missed = ['ALLOW']

chunk_size=10
chunked_words_list = [words_list[f:f+chunk_size] for f in range(0, len(words_list), chunk_size)]
temp_words_list_outer = chunked_words_list[:]
for temp_words_list in temp_words_list_outer:
    for temp_word in temp_words_list:
        print("Current Word: {}".format(temp_word))
        videos = glob.glob(os.path.join(lrw_path, temp_word, which_folder, '*.mp4'))
        for video in tqdm(videos):
            landmarks_extract_save(video, landmarks_path)

Current Word: ABOUT


100%|██████████| 1000/1000 [00:00<00:00, 117517.13it/s]


Current Word: ABSOLUTELY


100%|██████████| 959/959 [00:00<00:00, 159092.57it/s]


Current Word: ABUSE


...

100%|██████████| 1000/1000 [00:00<00:00, 201571.70it/s]


Current Word: YESTERDAY


100%|██████████| 1000/1000 [00:00<00:00, 233626.91it/s]


Current Word: YOUNG


100%|██████████| 1000/1000 [00:00<00:00, 227419.83it/s]


In [16]:

for temp_word in words_list:
    print("Current Word: {}".format(temp_word))
    videos = glob.glob(os.path.join(lrw_path, temp_word, which_folder, '*.mp4'))
    for video in tqdm(videos):
        landmarks_extract_save(video, landmarks_path)

Current Word: ABOUT


100%|██████████| 1000/1000 [00:00<00:00, 192319.87it/s]


Current Word: ABSOLUTELY


100%|██████████| 959/959 [00:00<00:00, 228334.33it/s]


Current Word: ABUSE


...

100%|██████████| 1000/1000 [00:00<00:00, 196740.18it/s]


Current Word: YESTERDAY


100%|██████████| 1000/1000 [00:00<00:00, 195483.97it/s]


Current Word: YOUNG


100%|██████████| 1000/1000 [00:00<00:00, 194117.83it/s]
